# **SMILES 文字列生成のための変分オートエンコーダ（VAE）**
本プロジェクトでは、**SMILES（Simplified Molecular Input Line Entry System）** という化学構造式の表記法を対象に、**変分オートエンコーダ（VAE）** を用いて学習・生成するモデルを構築しました。以下に、全体の処理の流れをまとめます。

---

## **1. ライブラリのインポートと環境設定**
```python
import torch
import numpy as np
import pandas as pd
import os
from torch.utils.data import TensorDataset, DataLoader
from rdkit import Chem, RDLogger
from google.colab import drive
```
- **Google Drive をマウント** し、学習データを取得。
- **RDKit（化学分子処理ライブラリ）** を使用し、ログを無効化。

---

## **2. データの準備**
```python
directory = '/content/drive/My Drive/day6'
property_df = pd.read_csv(os.path.join(directory, 'property_df.csv'))
```
- Google Drive 内の CSV ファイルから **SMILES データを読み込み**。

```python
vocab_freq = {}
word_length_dist = []
for smile in property_df["Open Babel SMILES"]:
    for s in smile:
        vocab_freq[s] = vocab_freq.get(s, 0) + 1
    word_length_dist.append(len(smile))

vocab = list(vocab_freq.keys())
N_INPUT = len(vocab) * 128
```
- **SMILES 文字列のトークンを解析し、語彙（vocab）を作成**。
- **最大文字長を 128 に統一**。

---

## **3. SMILES 文字列を数値ベクトル化**
```python
def smile2vec(vocab, vecsize, smile):
    vec = []
    for i in range(vecsize):
        v = [0 for _ in range(len(vocab))]
        if i < len(smile):
            v[vocab.index(smile[i])] = 1
        vec += v
    return vec
```
- SMILES の各文字を **One-hot ベクトルに変換** し、固定長 `vecsize=128` にする。

```python
X = np.array([smile2vec(vocab, 128, smile) for smile in property_df["Open Babel SMILES"]])
X_tensor = torch.from_numpy(X).float()
dataset = TensorDataset(X_tensor)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
```
- **データセットを PyTorch 用に変換し、DataLoader に格納**。

---

## **4. 変分オートエンコーダ（VAE）の実装**
```python
class VAE(torch.nn.Module):
    def __init__(self, n_input=784, n_hidden=400, n_z=20):
        super(VAE, self).__init__()
        self.fc1 = torch.nn.Linear(n_input, n_hidden)
        self.fc2 = torch.nn.Linear(n_hidden, n_z)  # μ
        self.fc3 = torch.nn.Linear(n_hidden, n_z)  # log(σ^2)
        self.fc4 = torch.nn.Linear(n_z, n_hidden)
        self.fc5 = torch.nn.Linear(n_hidden, n_input)

    def encode(self, x):
        h = torch.nn.functional.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)  # μ, log(σ^2)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std  # 再パラメータ化

    def decode(self, z):
        h = torch.nn.functional.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h))  # 生成データ

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        return self.decode(z), mu, log_var
```
- **エンコーダ**: `x → h → (μ, log(σ^2))`
- **再パラメータ化**: `z = μ + εσ`
- **デコーダ**: `z → h → x'`
- **出力**: `x'`（再構成された SMILES）

---

## **5. 損失関数と学習**
```python
model = VAE(n_input=N_INPUT, n_hidden=400, n_z=20).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
```
- **Adam オプティマイザで学習**。

```python
for epoch in range(20):
    for i, x in enumerate(data_loader):
        x = x[0].to(device).view(-1, N_INPUT)
        x_reconst, mu, log_var = model(x)

        reconst_loss = torch.nn.functional.binary_cross_entropy(x_reconst, x, reduction='sum')
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss = reconst_loss + kl_div

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
```
- **再構成誤差（Binary Cross Entropy）** + **KL ダイバージェンス** を最小化。

```python
    with torch.no_grad():
        z = torch.randn(16, 20).to(device)
        out = model.decode(z)
        print("Epoch[{}/{}], Generated SMILES: {}".format(epoch+1, 20, get_best_smile(out)))
```
- **学習後に新しい SMILES を生成**。

---

## **6. SMILES の復元関数**
```python
def get_best_smile(out_tensor):
    best_smile = ""
    for vec in out_tensor:
        vec = vec.reshape(128, len(vocab))
        smile = "".join([vocab[torch.argmax(v).item()] for v in vec])
        mol = Chem.MolFromSmiles(smile)

        while not mol:
            if len(smile) == 0: break
            smile = smile[:-1]
            mol = Chem.MolFromSmiles(smile)

        if len(best_smile) < len(smile):
            best_smile = smile
    return best_smile
```
- **One-hot ベクトル → SMILES 文字列に変換**。
- **無効な SMILES は修正**（末尾から1文字ずつ削除）。

---

# **まとめ**
本プロジェクトでは、**変分オートエンコーダ（VAE）を用いた SMILES 文字列の生成** を行いました。

### **実装のポイント**
1. **SMILES データの前処理**
   - One-hot エンコーディングで固定長ベクトル化。
2. **VAE モデルの設計**
   - エンコーダで `μ, log(σ^2)` を求め、再パラメータ化。
   - デコーダで `z` から SMILES を再構成。
3. **損失関数**
   - `BCE`（再構成誤差）+ `KL ダイバージェンス`。
4. **学習**
   - PyTorch の `Adam` オプティマイザを使用。
5. **生成**
   - 学習後、`z ~ N(0,1)` から新しい SMILES を生成。

### **期待される応用**
- **新規化合物の探索**
- **分子生成モデルの構築**
- **化学構造のデータ拡張**

VAE による分子生成は **新しい医薬品や材料の設計** にも応用可能であり、今後の発展が期待される技術です！

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

このコードは、PyTorch を用いて計算を行うデバイス（CPU または GPU）を自動的に選択する処理を行っています。以下に詳しく解説します。

---

### **コードの解説**
```python
import torch
```
- PyTorch のライブラリをインポートします。

```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
```
- `torch.cuda.is_available()` は、CUDA に対応した GPU が使用可能かどうかを判定する関数です。
  - GPU が使用可能な場合 (`True`)、`'cuda'` を返します。
  - GPU が使用できない場合 (`False`)、`'cpu'` を返します。
- `torch.device()` は、テンソルやモデルをどのデバイス（CPU または GPU）に配置するかを指定するためのオブジェクトを作成します。
- 結果として、GPU が利用可能であれば `device` には `'cuda'` が設定され、GPU が使えない場合は `'cpu'` が設定されます。

```python
device
```
- `device` を出力すると、現在選択されているデバイス（`'cuda'` または `'cpu'`）が表示されます。

---

### **このコードの用途**
- PyTorch を用いた機械学習・深層学習のプログラムでは、計算を GPU で行うことで高速化できます。
- このコードを使うことで、GPU が使える環境では自動的に GPU を使用し、そうでない場合は CPU を使用するようにできます。

---

### **実際の使用例**
例えば、モデルやテンソルを `device` に適用することで、適切なデバイス上で計算を行えます。

```python
# デバイスの選択
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# デバイス情報の表示
print(f'Using device: {device}')

# Tensorをデバイスに移動
x = torch.tensor([1.0, 2.0, 3.0]).to(device)
print(x.device)  # 選択されたデバイスが表示される
```

---

### **まとめ**
- `torch.device('cuda' if torch.cuda.is_available() else 'cpu')` は、利用可能な計算デバイス（GPU か CPU）を自動的に選択する。
- GPU が使える場合は `cuda`、使えない場合は `cpu` が選択される。
- `device` を指定してテンソルやモデルを適切なデバイスに移動させることで、計算を最適化できる。

このコードは、PyTorch を使う際の基本的なベストプラクティスの一つなので、覚えておくと便利です！

In [ ]:
from google.colab import drive
import pandas as pd
import os

drive.mount('/content/drive')
directory = '/content/drive/My Drive/day6'
property_df = pd.read_csv(os.path.join(directory, 'property_df.csv'))

Mounted at /content/drive


このコードは、Google Colab で Google ドライブをマウントし、指定したディレクトリ内の CSV ファイルを `pandas` のデータフレームとして読み込む処理を行っています。以下、詳しく解説します。

---

## **コードの解説**

### **1. Google ドライブのマウント**
```python
from google.colab import drive
```
- Google Colab で Google ドライブを操作するための `drive` モジュールをインポートします。

```python
drive.mount('/content/drive')
```
- Google ドライブを `/content/drive` にマウントします。
- 実行すると、認証を求められるので、表示されるリンクを開き、Google アカウントでログインして認証コードをコピーして貼り付ける必要があります。
- マウント後、Google ドライブのファイルにローカルのようにアクセスできるようになります。

---

### **2. `pandas` と `os` のインポート**
```python
import pandas as pd
import os
```
- `pandas` はデータ分析のためのライブラリで、CSV ファイルの読み込みなどに使用します。
- `os` はファイルパスの結合 (`os.path.join`) など、OS に依存した操作をするための標準ライブラリです。

---

### **3. データの読み込み**
```python
directory = '/content/drive/My Drive/day6'
```
- Google ドライブ内の `"My Drive/day6"` フォルダのパスを指定します。
- `My Drive` は Google ドライブのルートディレクトリに相当します。

```python
property_df = pd.read_csv(os.path.join(directory, 'property_df.csv'))
```
- `os.path.join(directory, 'property_df.csv')` によって `"My Drive/day6/property_df.csv"` というフルパスを作成します。
- `pd.read_csv()` を使い、指定した CSV ファイルをデータフレーム (`property_df`) として読み込みます。

---

## **補足**
1. **Google ドライブ内のファイルを使う理由**
   - Colab は仮想環境のため、ランタイムをリセットするとローカルファイルが消えます。
   - Google ドライブにデータを保存しておくと、再実行時にファイルを再利用できます。

2. **Google ドライブ内のファイルパスの確認**
   - Google ドライブをマウントした後、次のコマンドでドライブ内のフォルダやファイルを確認できます。
     ```python
     !ls /content/drive/My\ Drive/day6
     ```
   - もし `FileNotFoundError` が発生したら、指定した `directory` にファイルが存在するかを確認してください。

---

## **まとめ**
- Google ドライブをマウントして Colab でファイルを扱えるようにする。
- `pd.read_csv()` を使い、Google ドライブ内の CSV ファイルを `pandas` のデータフレームとして読み込む。
- `os.path.join()` を使うことで OS に依存しないパスを作成できる。

この方法を使えば、大量のデータを Colab で効率的に扱うことができます！

In [ ]:
SMILES_COL = "Open Babel SMILES"
SMILES_MAXLEN = 128
BATCH_SIZE = 16
N_HIDDEN = 400
N_Z = 20
LEARNING_RATE = 1e-3
NUM_EPOCHS = 20

vocab_freq =  {}
word_length_dist = []
for smile in property_df[SMILES_COL]:
    for s in smile:
        if s not in vocab_freq.keys():
            vocab_freq[s] = 0
        vocab_freq[s] += 1
    word_length_dist.append(len(smile))

vocab = list(vocab_freq.keys())
N_INPUT = len(vocab)*SMILES_MAXLEN

このコードは、**SMILES（Simplified Molecular Input Line Entry System）表記の分子データを処理し、文字の頻度分布を作成する前処理**を行っています。主に、SMILES データの特徴を抽出し、ニューラルネットワークの入力サイズを計算するために使用されます。

---

## **コードの解説**

### **1. ハイパーパラメータの定義**
```python
SMILES_COL = "Open Babel SMILES"
SMILES_MAXLEN = 128
BATCH_SIZE = 16
N_HIDDEN = 400
N_Z = 20
LEARNING_RATE = 1e-3
NUM_EPOCHS = 20
```
- **`SMILES_COL`**: SMILES データが格納されている `property_df` のカラム名を指定。
- **`SMILES_MAXLEN`**: SMILES 文字列の最大長。短いものはパディングされる可能性がある。
- **`BATCH_SIZE`**: 学習時のバッチサイズ（1 回の更新で処理するデータ数）。
- **`N_HIDDEN`**: 隠れ層のニューロン数（ニューラルネットワークの層のサイズ）。
- **`N_Z`**: 潜在変数（潜在空間の次元数、VAE などの生成モデルで使う）。
- **`LEARNING_RATE`**: 学習率（勾配降下法の更新幅）。
- **`NUM_EPOCHS`**: エポック数（データセット全体を何回学習するか）。

---

### **2. SMILES データの頻度分析**
```python
vocab_freq = {}
word_length_dist = []
```
- **`vocab_freq`**: SMILES 文字（原子記号や結合情報など）の出現回数を記録する辞書。
- **`word_length_dist`**: 各 SMILES 文字列の長さを保存するリスト。

```python
for smile in property_df[SMILES_COL]:  # 各 SMILES 文字列に対して
    for s in smile:  # 文字ごとにループ
        if s not in vocab_freq.keys():  # 初めて出現する文字なら辞書に追加
            vocab_freq[s] = 0
        vocab_freq[s] += 1  # 出現回数をカウント
    word_length_dist.append(len(smile))  # 文字列の長さを保存
```
- **`vocab_freq` の役割**
  - すべての SMILES 文字列をスキャンし、各文字（原子記号、結合記号など）が何回登場するかをカウントする。
  - 例: `vocab_freq = {'C': 500, 'O': 200, '=': 100, ...}`

- **`word_length_dist` の役割**
  - 各 SMILES の長さを記録し、長さの統計を取るために使用。
  - 例: `word_length_dist = [25, 30, 22, 28, ...]`

---

### **3. 語彙リストと入力次元の計算**
```python
vocab = list(vocab_freq.keys())  # ユニークな SMILES 文字のリスト
N_INPUT = len(vocab) * SMILES_MAXLEN  # 入力次元数
```
- **`vocab`**: 見つかったユニークな文字をリスト化（辞書のキーを取得）。
  - 例: `vocab = ['C', 'O', 'N', '=', '#', '(', ')', ...]`
  - このリストの長さが、1-hot エンコーディングで使う語彙サイズになる。

- **`N_INPUT`**: ニューラルネットワークの入力次元を計算。
  - 文字の種類 `len(vocab)` × 文字列の最大長 `SMILES_MAXLEN`
  - 例: `N_INPUT = 50 * 128 = 6400`
  - SMILES を 1-hot ベクトルでエンコードする場合、この値がニューラルネットの入力サイズになる。

---

## **コードの目的**
このコードの主な目的は、**SMILES 文字列の前処理と、ニューラルネットワークの入力サイズの決定**です。
- **文字ごとの頻度を計算** → SMILES を効率的に処理できるようにする。
- **語彙リストを作成** → ニューラルネットの入力を定義。
- **入力次元を計算** → モデルの構造を設計する際に必要。

---

## **応用例**
このデータは、**ニューラルネットワーク（VAE や RNN など）を使った分子生成・予測タスク** に利用できます。
- **VAE（Variational Autoencoder）**: SMILES を潜在空間にマッピングして、新しい分子を生成。
- **RNN（Recurrent Neural Network）**: SMILES 文字列を時系列データとして処理し、分子の特性を予測。

---

## **まとめ**
1. **ハイパーパラメータの設定**
   - SMILES の最大長、バッチサイズ、学習率などを決める。
2. **SMILES 文字ごとの出現回数を記録**
   - `vocab_freq` を作成し、どの文字がどれだけ使われているかをカウント。
3. **語彙リストを作成**
   - `vocab` を作り、ニューラルネットの入力に必要な次元 `N_INPUT` を計算。

この前処理を行うことで、SMILES を扱う機械学習モデルに適したデータを準備できます！

In [ ]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

def smile2vec(vocab, vecsize, smile):
    vec = []
    for i in range(vecsize):
        v = [0 for _ in range(len(vocab))]
        if i < len(smile):
            v[vocab.index(smile[i])] = 1
        vec += v
    return vec


X = np.array([smile2vec(vocab, SMILES_MAXLEN, smile) for smile in list(property_df[SMILES_COL])])
X_tensor = torch.from_numpy(X).float()
dataset = TensorDataset(X_tensor)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

このコードは、**SMILES（分子の文字列表現）を数値ベクトルに変換し、それを PyTorch の `DataLoader` に格納する**ための処理を行っています。主に以下のステップで構成されています。

---

## **コードの解説**

### **1. 必要なライブラリのインポート**
```python
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
```
- **`numpy`**: データの変換に使用。
- **`torch`**: PyTorch のテンソル操作に使用。
- **`TensorDataset`**: PyTorch でデータセットを作成するためのクラス。
- **`DataLoader`**: データをバッチごとに処理し、シャッフルなどを適用するためのクラス。

---

### **2. SMILES をベクトルに変換する関数**
```python
def smile2vec(vocab, vecsize, smile):
    vec = []
    for i in range(vecsize):
        v = [0 for _ in range(len(vocab))]  # 語彙サイズの長さを持つ 0 ベクトルを作成
        if i < len(smile):  # SMILES 文字列の長さを超えない範囲で
            v[vocab.index(smile[i])] = 1  # 文字を one-hot encoding
        vec += v  # フラットなリストとして格納
    return vec
```
#### **この関数の処理**
- **`vocab`**: SMILES に含まれるユニークな文字のリスト。
- **`vecsize`**: ベクトルの最大長（`SMILES_MAXLEN`）。
- **`smile`**: 入力の SMILES 文字列。

#### **処理の流れ**
1. `vec = []` を用意（ベクトルを格納するリスト）。
2. `vecsize` の長さ (`SMILES_MAXLEN` など) だけループする。
3. `vocab` のサイズと同じ長さの 0 ベクトル `v` を作成。
4. 文字列 `smile` の範囲内なら、その文字の位置を 1 にする（**one-hot encoding**）。
5. `v` を `vec` に追加（フラットなリストとして格納）。

#### **例**
```python
vocab = ['C', 'O', 'N', '=', '(']
smile = "COO"
vecsize = 5
smile2vec(vocab, vecsize, smile)
```
- `C` → `[1, 0, 0, 0, 0]`
- `O` → `[0, 1, 0, 0, 0]`
- `O` → `[0, 1, 0, 0, 0]`
- パディング部分 → `[0, 0, 0, 0, 0]`
- **結果**
```python
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
```
このように、**SMILES を固定長のベクトルに変換**する。

---

### **3. SMILES データをベクトル化**
```python
X = np.array([smile2vec(vocab, SMILES_MAXLEN, smile) for smile in list(property_df[SMILES_COL])])
```
- **`property_df[SMILES_COL]`** に含まれるすべての SMILES を `smile2vec()` に適用。
- `numpy.array()` を使い、リストを NumPy 配列に変換。

---

### **4. NumPy 配列を PyTorch のテンソルに変換**
```python
X_tensor = torch.from_numpy(X).float()
```
- `torch.from_numpy(X)` によって NumPy 配列を PyTorch のテンソルに変換。
- `.float()` を適用し、データ型を `float32` に変換（PyTorch のモデルは `float32` を使用することが多いため）。

---

### **5. PyTorch のデータセットとデータローダーの作成**
```python
dataset = TensorDataset(X_tensor)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
```
- `TensorDataset(X_tensor)` を作成（`X_tensor` を PyTorch のデータセット形式に変換）。
- `DataLoader()` を用いて、データセットをバッチごとに分割し、シャッフルを適用。
  - **`batch_size=BATCH_SIZE`**: バッチサイズ（例: 16）。
  - **`shuffle=True`**: データをランダムに並び替え（学習のバイアスを防ぐ）。

---

## **コードの目的**
このコードは、**SMILES 文字列をニューラルネットワークで扱いやすい形式（固定長ベクトル）に変換し、PyTorch の `DataLoader` を使ってバッチ処理を可能にする**ことが目的です。

### **流れを整理**
1. **SMILES を one-hot ベクトルに変換**（`smile2vec()`）。
2. **NumPy 配列に変換**。
3. **PyTorch のテンソルに変換**。
4. **データセットとデータローダーを作成**。

---

## **応用**
この手法は、**深層学習モデル（例えば RNN, Transformer, VAE）を用いた分子生成・特性予測** に利用できます。
- **VAE（Variational Autoencoder）**: SMILES を潜在空間に圧縮し、新しい分子を生成。
- **RNN（Recurrent Neural Network）**: SMILES の時系列データとしての性質を活かし、分子の特性を予測。
- **Transformer**: より高性能な分子表現を学習し、分子の生成や特性予測を行う。

---

## **まとめ**
- `smile2vec()` で **SMILES を one-hot ベクトル化**。
- NumPy 配列に変換し、**PyTorch のテンソルに変換**。
- `TensorDataset` と `DataLoader` を用いて、**ミニバッチ学習が可能なデータローダーを作成**。
- **ニューラルネットワークの入力データを準備する前処理**として非常に重要。

この手法を応用すれば、SMILES から深層学習を用いた分子生成・特性予測が可能になります！

In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 48.4 MB/s eta 0:00:00


このコードは、Python のパッケージマネージャー `pip` を使用して **RDKit** をインストールするためのコマンドです。  

```python
!pip install rdkit
```
- **`!`（エクスクラメーションマーク）**  
  - Jupyter Notebook や Google Colab などの環境では、`!` を先頭につけることで、Python ではなく **シェルコマンド**（ターミナルで実行するコマンド）を実行できます。
  - つまり、`pip install rdkit` というシェルコマンドを Python の環境から実行している。

- **`pip install rdkit`**  
  - `pip` は Python のパッケージ管理ツールで、外部ライブラリのインストールに使用。
  - `rdkit` は、化学情報学（ケモインフォマティクス）のための Python ライブラリで、分子の構造解析やシミュレーションに利用される。

---

## **RDKit とは？**
**RDKit（The Open-Source Cheminformatics Toolkit）** は、**化学情報学**（Chemoinformatics）に特化したオープンソースのツールキットです。  
このライブラリを使うことで、以下のような処理が可能になります。

### **主な機能**
1. **SMILES（分子表記）と分子構造の変換**
   ```python
   from rdkit import Chem
   mol = Chem.MolFromSmiles('CCO')  # エタノールの分子構造を作成
   print(mol)
   ```
   → SMILES から分子オブジェクトを生成。

2. **分子の可視化**
   ```python
   from rdkit.Chem import Draw
   Draw.MolToImage(mol)
   ```
   → 分子の構造式を描画。

3. **分子の特徴量抽出**
   ```python
   from rdkit.Chem import Descriptors
   mw = Descriptors.MolWt(mol)  # 分子量を計算
   print(mw)
   ```
   → 分子量や LogP（脂溶性）などの計算。

4. **分子のフィンガープリント（分子の特徴を数値ベクトルに変換）**
   ```python
   from rdkit.Chem import rdMolDescriptors
   fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
   print(fp)
   ```
   → 分子を機械学習で扱うために、数値ベクトル（フィンガープリント）に変換。

---

## **RDKit のインストール時の注意点**
1. **Google Colab の場合**
   - `rdkit` は通常の `pip install` では動作しないことがあるため、以下のように `conda` を使うのが一般的：
     ```python
     !conda install -c conda-forge rdkit -y
     ```
   - もしくは、Colab では以下の方法も推奨される：
     ```python
     !pip install rdkit-pypi
     ```

2. **ローカル環境（Windows / Mac）**
   - 公式サイト（https://www.rdkit.org/）にある手順に従って **Anaconda 環境** でのインストールが推奨される。
   - 例：
     ```bash
     conda install -c conda-forge rdkit
     ```

---

## **まとめ**
- `!pip install rdkit` は、RDKit を Python 環境にインストールするコマンド。
- RDKit は **SMILES の処理、分子の可視化、特徴量抽出** などが可能な **化学情報学のライブラリ**。
- Google Colab では `!pip install rdkit-pypi` や `!conda install -c conda-forge rdkit` が推奨されることがある。

RDKit を使えば、機械学習による分子設計や特性予測などが可能になります！ 🚀

In [ ]:
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

def get_best_smile(out_tensor):
    best_smile = ""
    for vec in out_tensor:
        vec = vec.reshape(SMILES_MAXLEN, len(vocab))
        smile = "".join([vocab[torch.argmax(v).item()] for v in vec])
        mol = Chem.MolFromSmiles(smile)
        while not mol:
            if len(smile) == 0: break
            smile = smile[:-1]
            mol = Chem.MolFromSmiles(smile)

        if len(best_smile) < len(smile):
            best_smile = smile

    return best_smile


このコードは、**RDKit を使用してニューラルネットワークの出力テンソルを SMILES 文字列に変換し、最も妥当な SMILES（有効な化学構造）を取得する関数** `get_best_smile()` を定義しています。

---

## **コードの解説**

### **1. 必要なライブラリのインポート**
```python
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
```
- `from rdkit import Chem`  
  - RDKit の `Chem` モジュールをインポートし、分子構造の解析や変換に使用。
- `from rdkit import RDLogger`
  - RDKit の **ログ出力を制御** するためのモジュール。
- `RDLogger.DisableLog('rdApp.*')`
  - RDKit の警告ログを非表示にする（エラーメッセージを減らし、コードを見やすくする）。

---

### **2. `get_best_smile()` 関数の定義**
```python
def get_best_smile(out_tensor):
```
この関数は、**ニューラルネットワークの出力テンソル（`out_tensor`）を SMILES 文字列に変換し、有効な構造の中で最も長い SMILES を返す**。

#### **引数**
- `out_tensor`:  
  - **モデルの出力テンソル**。  
  - このテンソルは、one-hot エンコーディングされた SMILES に対応している。

#### **返り値**
- **最も長い有効な SMILES 文字列**。

---

### **3. 変数の初期化**
```python
best_smile = ""
```
- `best_smile` には **最も長い有効な SMILES 文字列** を格納。

---

### **4. テンソルを SMILES に変換**
```python
for vec in out_tensor:
    vec = vec.reshape(SMILES_MAXLEN, len(vocab))
    smile = "".join([vocab[torch.argmax(v).item()] for v in vec])
```
#### **処理の流れ**
1. `out_tensor` の各要素 `vec` に対してループ。
2. `vec.reshape(SMILES_MAXLEN, len(vocab))`  
   - `vec` を (SMILES 文字列の最大長, 語彙サイズ) の形にリシェイプ。
   - つまり、**SMILES 文字列の各文字が one-hot エンコーディングされた状態**になる。
3. `torch.argmax(v).item()` を使って、**各位置で最も確率が高い（最大値を持つ）インデックスを取得**。
4. `vocab` から対応する文字を取得して `smile` を作成。

---

### **5. 生成された SMILES が有効かチェック**
```python
mol = Chem.MolFromSmiles(smile)
while not mol:
    if len(smile) == 0: break
    smile = smile[:-1]
    mol = Chem.MolFromSmiles(smile)
```
- `Chem.MolFromSmiles(smile)`  
  - **RDKit の関数**で、SMILES 文字列 `smile` から分子オブジェクト（`mol`）を生成。
  - `mol` が `None` の場合、その SMILES は無効（化学的に正しくない）。
- **無効な場合の処理**：
  - `while not mol:` で無効な場合は、末尾の文字を削除（`smile = smile[:-1]`）。
  - これを繰り返して **有効な SMILES になるまで修正**。
  - `if len(smile) == 0: break` で、全削除される場合の対策。

---

### **6. 最も長い有効な SMILES を選択**
```python
if len(best_smile) < len(smile):
    best_smile = smile
```
- 現在の `smile` の長さが、`best_smile` より長い場合、更新。
- **最も長い有効な SMILES を保持**。

---

### **7. 最終的な SMILES を返す**
```python
return best_smile
```
- **最も長い有効な SMILES を関数の出力として返す**。

---

## **コードの目的**
- **ニューラルネットワークの出力テンソルを、適切な SMILES 文字列に変換**。
- **無効な SMILES を修正し、最も長い有効な SMILES を選択**。

---

## **具体例**
### **入力（`out_tensor`）**
仮に `out_tensor` の 1 つの要素が次のような形だったとする：
```python
out_tensor[0] = [
  [0, 1, 0, 0, 0],  # 'C'
  [0, 0, 1, 0, 0],  # 'O'
  [0, 0, 1, 0, 0],  # 'O'
  [0, 0, 0, 0, 1],  # '('
  [0, 0, 0, 0, 0],  # パディング
]
```
対応する `vocab`:
```python
vocab = ['C', 'O', 'N', '=', '(']
```
### **処理**
- `torch.argmax(v).item()` を使い、最も大きな値のインデックスを取得。
- `vocab` を参照して `smile` を作成：
  ```python
  smile = "COO("
  ```
- `Chem.MolFromSmiles(smile)` で **RDKit による分子のバリデーション**。
  - `"COO("` は無効なので、`"COO"` まで削除し、`mol` が作成可能な形にする。

### **最終出力**
`best_smile = "COO"`

---

## **まとめ**
- **`out_tensor` を SMILES 文字列に変換**：
  - **one-hot ベクトル → SMILES 文字列** にデコード。
- **無効な SMILES を修正**：
  - RDKit (`Chem.MolFromSmiles()`) を使い、無効な構造を削除。
- **最も長い有効な SMILES を返す**：
  - ニューラルネットワークが生成する SMILES から **最も良い分子表現を取得**。

この関数は **分子生成モデル（例：VAE, GAN）** などで、**生成された SMILES をクリーンな形式に変換する際に役立つ**！ 🚀

In [ ]:
class VAE(torch.nn.Module):
    def __init__(self, n_input=784, n_hidden=400, n_z=20):
        super(VAE, self).__init__()
        self.fc1 = torch.nn.Linear(n_input, n_hidden)
        self.fc2 = torch.nn.Linear(n_hidden, n_z)
        self.fc3 = torch.nn.Linear(n_hidden, n_z)
        self.fc4 = torch.nn.Linear(n_z, n_hidden)
        self.fc5 = torch.nn.Linear(n_hidden, n_input)

    def encode(self, x):
        h = torch.nn.functional.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = torch.nn.functional.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h))

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

このコードは、**変分オートエンコーダ（VAE: Variational Autoencoder）** を PyTorch で実装したものです。  
VAE は、データを圧縮（エンコード）し、潜在空間で表現を学習しながら、新しいデータを生成（デコード）するためのニューラルネットワークです。

---

## **クラスの概要**
```python
class VAE(torch.nn.Module):
```
- PyTorch の `torch.nn.Module` を継承して、ニューラルネットワークモデルを定義。
- **変分オートエンコーダ（VAE）** を構築。

---

## **1. `__init__()`（初期化関数）**
```python
def __init__(self, n_input=784, n_hidden=400, n_z=20):
```
- `n_input`: 入力の次元数（デフォルト: 784）  
  - 例：`28×28` の画像（MNIST）の場合、784次元。
- `n_hidden`: 隠れ層の次元数（デフォルト: 400）
- `n_z`: 潜在変数（潜在ベクトル `z`）の次元数（デフォルト: 20）

### **エンコーダ部分（Encoder）**
```python
self.fc1 = torch.nn.Linear(n_input, n_hidden)  # 入力 → 隠れ層
self.fc2 = torch.nn.Linear(n_hidden, n_z)  # 潜在変数の平均（μ）
self.fc3 = torch.nn.Linear(n_hidden, n_z)  # 潜在変数の対数分散（log(σ²)）
```
- `fc1`: 入力データを `n_hidden` の次元に変換する全結合層（Encoder の隠れ層）。
- `fc2`: **潜在変数の平均** `μ` を出力。
- `fc3`: **潜在変数の対数分散** `log(σ²)` を出力。

### **デコーダ部分（Decoder）**
```python
self.fc4 = torch.nn.Linear(n_z, n_hidden)  # 潜在変数 → 隠れ層
self.fc5 = torch.nn.Linear(n_hidden, n_input)  # 隠れ層 → 再構成データ
```
- `fc4`: 潜在変数 `z` を `n_hidden` の次元に変換する全結合層。
- `fc5`: `n_hidden` を `n_input` に変換し、元のデータを復元。

---

## **2. `encode()`（エンコーダ）**
```python
def encode(self, x):
    h = torch.nn.functional.relu(self.fc1(x))
    return self.fc2(h), self.fc3(h)
```
- `x` をエンコーダに通して、隠れ層を計算（`fc1` → ReLU 活性化関数）。
- **潜在変数の平均 (`μ`) と対数分散 (`log_var`) を出力**。

---

## **3. `reparameterize()`（再パラメータ化トリック）**
```python
def reparameterize(self, mu, log_var):
    std = torch.exp(log_var / 2)  # 分散の平方根（標準偏差）
    eps = torch.randn_like(std)  # 標準正規分布からサンプリング
    return mu + eps * std
```
- `log_var` を指数変換し、標準偏差 `std = exp(log_var/2)` を計算。
- `torch.randn_like(std)` で標準正規分布 `N(0,1)` から乱数 `ε` をサンプリング。
- **サンプリングした `ε` に `μ` を加え、最終的な `z` を作成**：
  \[
  z = \mu + \epsilon \cdot \sigma
  \]
  これを **再パラメータ化トリック（Reparameterization Trick）** と呼び、勾配計算を可能にする。

---

## **4. `decode()`（デコーダ）**
```python
def decode(self, z):
    h = torch.nn.functional.relu(self.fc4(z))
    return torch.sigmoid(self.fc5(h))
```
- `z` をデコーダに通して、元のデータの復元を試みる。
- `fc4(z)` を `ReLU` で活性化。
- `fc5` を通し、`sigmoid` 関数で出力（**値を 0 〜 1 に制約**）。

---

## **5. `forward()`（順伝播）**
```python
def forward(self, x):
    mu, log_var = self.encode(x)  # 1. エンコーダ
    z = self.reparameterize(mu, log_var)  # 2. 潜在変数のサンプリング
    x_reconst = self.decode(z)  # 3. デコーダ
    return x_reconst, mu, log_var  # 4. 再構成データと潜在変数を返す
```
1. **エンコーダで `μ` と `log_var` を計算**。
2. **再パラメータ化トリックで `z` をサンプリング**。
3. **デコーダで `z` から元データを復元**。
4. **再構成データ `x_reconst`、`μ`、`log_var` を返す**。

---

## **VAE の全体像**
### **エンコーダ（Encoder）**
データ **x** を潜在空間の確率分布に変換：
\[
q(z|x) = \mathcal{N}(\mu, \sigma^2)
\]

### **再パラメータ化**
潜在変数 `z` をサンプリング：
\[
z = \mu + \epsilon \cdot \sigma, \quad \epsilon \sim \mathcal{N}(0,1)
\]

### **デコーダ（Decoder）**
`z` から元データを復元：
\[
p(x|z)
\]

---

## **損失関数（VAE の学習）**
通常のオートエンコーダとは異なり、**VAE では 2 つの損失を最小化**：
1. **再構成誤差（MSE や BCE）**  
   - `x_reconst` と `x` の誤差（ピクセルごとの違い）。
2. **KL ダイバージェンス（正則化項）**  
   - 潜在変数 `z` の分布を、標準正規分布 `N(0,1)` に近づける。

損失関数：
\[
\mathcal{L} = \text{Reconstruction Loss} + \beta \cdot D_{KL}(q(z|x) || p(z))
\]
\[
D_{KL} = -\frac{1}{2} \sum_{i} \left( 1 + \log\sigma_i^2 - \mu_i^2 - \sigma_i^2 \right)
\]

---

## **まとめ**
- **VAE（変分オートエンコーダ）** を PyTorch で実装。
- **`encode()`**: `x` を潜在変数の分布 `(μ, log_var)` に変換。
- **`reparameterize()`**: `z = μ + εσ` を計算（再パラメータ化）。
- **`decode()`**: `z` からデータを再構成。
- **`forward()`**: エンコーダ → 潜在変数 → デコーダの流れを処理。

この VAE を使うことで、**画像生成や分子生成（SMILES 文字列）** などの応用が可能！ 🚀

In [ ]:
model = VAE(n_input=N_INPUT, n_hidden=N_HIDDEN, n_z=N_Z).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
losses = []
reconst_losses = []
kl_divs = []
for epoch in range(NUM_EPOCHS):
    for i, x in enumerate(data_loader):
        x = x[0].to(device).view(-1, N_INPUT)
        x_reconst, mu, log_var = model(x)

        reconst_loss = torch.nn.functional.binary_cross_entropy(x_reconst, x, reduction='sum')
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        reconst_losses.append(reconst_loss.item())
        kl_divs.append(kl_div.item())

        if (i+1) % 100 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}"
                   .format(epoch+1, NUM_EPOCHS, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))

    with torch.no_grad():
        z = torch.randn(BATCH_SIZE, N_Z).to(device)
        out = model.decode(z)
        print("Epoch[{}/{}], Generated SMILES: {}".format(epoch+1, NUM_EPOCHS, get_best_smile(out)))

        # out, _, _ = model(x)
        # print("Epoch[{}/{}], Reconstructed SMILES: {}".format(epoch+1, NUM_EPOCHS, get_best_smile(out)))

Epoch[1/20], Step [100/548], Reconst Loss: 1398.6638, KL Div: 195.9709
Epoch[1/20], Step [200/548], Reconst Loss: 1187.0525, KL Div: 130.9270
Epoch[1/20], Step [300/548], Reconst Loss: 1116.9512, KL Div: 97.0856
Epoch[1/20], Step [400/548], Reconst Loss: 901.8925, KL Div: 135.3994
Epoch[1/20], Step [500/548], Reconst Loss: 752.6674, KL Div: 135.0882
Epoch[1/20], Generated SMILES: O=NN([C@H](c1ccccc1)OC(=O)C)CC
Epoch[2/20], Step [100/548], Reconst Loss: 716.5902, KL Div: 137.1727
Epoch[2/20], Step [200/548], Reconst Loss: 715.3192, KL Div: 132.9384
Epoch[2/20], Step [300/548], Reconst Loss: 772.6079, KL Div: 136.6139
Epoch[2/20], Step [400/548], Reconst Loss: 833.4825, KL Div: 119.4229
Epoch[2/20], Step [500/548], Reconst Loss: 741.7397, KL Div: 141.0988
Epoch[2/20], Generated SMILES: O=NN([C@H](c1ccccc1)OC(=O)C)CC
Epoch[3/20], Step [100/548], Reconst Loss: 777.6328, KL Div: 124.4297
Epoch[3/20], Step [200/548], Reconst Loss: 740.7799, KL Div: 126.4378
Epoch[3/20], Step [300/548], Recon

このコードは、**変分オートエンコーダ（VAE）** の学習と SMILES 文字列の生成を行う部分です。  
具体的には、以下の流れで処理が進みます：

1. **モデルと最適化手法の設定**
2. **学習ループ（エポックごとにミニバッチで学習）**
3. **損失（再構成誤差 + KL ダイバージェンス）の計算**
4. **逆伝播とパラメータ更新**
5. **定期的に生成した SMILES 文字列を表示**

---

## **コードの詳細解説**

### **1. モデルの作成と最適化手法の設定**
```python
model = VAE(n_input=N_INPUT, n_hidden=N_HIDDEN, n_z=N_Z).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
```
- `VAE` モデルのインスタンスを作成し、**GPU（または CPU）に送る** (`to(device)`)。
- **Adam** 最適化アルゴリズムを使用し、学習率 `LEARNING_RATE` でモデルのパラメータを更新。

---

### **2. 損失を記録するリストの作成**
```python
losses = []
reconst_losses = []
kl_divs = []
```
- **学習中の損失を記録するためのリスト** を用意：
  - `losses` → **合計損失**（再構成誤差 + KL ダイバージェンス）
  - `reconst_losses` → **再構成誤差**
  - `kl_divs` → **KL ダイバージェンス**

---

### **3. エポックごとの学習ループ**
```python
for epoch in range(NUM_EPOCHS):
```
- `NUM_EPOCHS` 回（例：20回）学習を繰り返す。

---

### **4. ミニバッチごとの学習処理**
```python
for i, x in enumerate(data_loader):
```
- `data_loader` からデータを **ミニバッチ** 単位で取得。
- `x` はバッチサイズ `BATCH_SIZE` のデータを含む。

```python
x = x[0].to(device).view(-1, N_INPUT)
```
- `x[0]` を GPU に転送し、形状を `(BATCH_SIZE, N_INPUT)` に変形。

---

### **5. 順伝播（フォワードパス）**
```python
x_reconst, mu, log_var = model(x)
```
- モデルに `x` を入力し、
  - **再構成データ `x_reconst`**
  - **潜在変数の平均 `mu`**
  - **潜在変数の対数分散 `log_var`**
  を取得。

---

### **6. 損失関数の計算**
```python
reconst_loss = torch.nn.functional.binary_cross_entropy(x_reconst, x, reduction='sum')
```
- **再構成誤差（Reconstruction Loss）**  
  - `x_reconst` と `x` の誤差を **バイナリークロスエントロピー（BCE）** で計算。
  - `reduction='sum'` → バッチ全体の誤差を合計。

```python
kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
```
- **KL ダイバージェンス（正則化項）**  
  - VAE の潜在変数 `z` の分布 `q(z|x)` を標準正規分布 `p(z) ~ N(0,1)` に近づけるための項。
  - 数式：
    \[
    D_{KL} = -\frac{1}{2} \sum_{i} \left( 1 + \log\sigma_i^2 - \mu_i^2 - \sigma_i^2 \right)
    \]

```python
loss = reconst_loss + kl_div
```
- **最終的な損失（合計損失）**  
  - 再構成誤差 `reconst_loss` と KL ダイバージェンス `kl_div` の合計。

---

### **7. 逆伝播とパラメータ更新**
```python
optimizer.zero_grad()  # 勾配の初期化
loss.backward()  # 逆伝播
optimizer.step()  # パラメータ更新
```
- **`zero_grad()`** → 前回の計算の勾配をクリア。
- **`backward()`** → 誤差逆伝播（Backpropagation）。
- **`step()`** → 最適化手法 `Adam` でパラメータを更新。

---

### **8. 損失の記録**
```python
losses.append(loss.item())
reconst_losses.append(reconst_loss.item())
kl_divs.append(kl_div.item())
```
- 計算した損失をリストに記録。

---

### **9. 100 バッチごとのログ表示**
```python
if (i+1) % 100 == 0:
    print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}"
           .format(epoch+1, NUM_EPOCHS, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
```
- **100 バッチごとに学習の進捗を表示**。

---

### **10. 各エポック後に生成 SMILES を表示**
```python
with torch.no_grad():
    z = torch.randn(BATCH_SIZE, N_Z).to(device)  # 標準正規分布から潜在変数をサンプリング
    out = model.decode(z)  # デコーダで新しいデータを生成
    print("Epoch[{}/{}], Generated SMILES: {}".format(epoch+1, NUM_EPOCHS, get_best_smile(out)))
```
- **`torch.no_grad()`**：勾配計算を無効化（メモリ節約）。
- **`z = torch.randn(BATCH_SIZE, N_Z)`**：標準正規分布 `N(0,1)` から `z` をサンプリング。
- **`model.decode(z)`**：デコーダで `z` から新しいデータ（SMILES）を生成。
- **`get_best_smile(out)`**：最も良い SMILES 文字列を抽出し、表示。

---

## **コードのポイント**
1. **VAE を用いて SMILES 文字列の学習を行う**
2. **エンコーダで潜在変数 (`z`) の分布を学習**
3. **デコーダで `z` からデータを生成**
4. **損失関数は「再構成誤差 + KL ダイバージェンス」**
5. **学習中に `z` から新しい SMILES 文字列を生成**

---

## **出力例**
```
Epoch[1/20], Step [100/500], Reconst Loss: 1567.2314, KL Div: 124.7845
Epoch[1/20], Step [200/500], Reconst Loss: 1432.1208, KL Div: 98.5342
...
Epoch[1/20], Generated SMILES: C1CCCCC1O
Epoch[2/20], Generated SMILES: CCOC(=O)C1=CC=CC=C1
...
```
- 生成される **SMILES 文字列** は、学習が進むにつれてより化学的に意味のある構造になる。

---

## **まとめ**
- **変分オートエンコーダ（VAE）を使って SMILES を学習し、生成するコード**。
- **再構成誤差（BCE）+ KL ダイバージェンス** を最小化することで、潜在空間を学習。
- **学習中に `z` から SMILES を生成** し、化学分子の生成ができる。